In [4]:
import pymongo
from pymongo import MongoClient
from pprint import pprint, pformat, PrettyPrinter
from bson.son import SON
from bson import ObjectId
from os.path import exists
import json
from ipy_table import *
from mongotools import *

In [5]:
# Connect to database and create variables for the collections
client = MongoClient()
db = client.lobbyradar
entities = db.entities
relations = db.relations

# Print some statistics about our database dump
print("Collections in database: " + ", ".join(db.collection_names(include_system_collections=False)))
print("Entities: %s documents" % entities.count())
print("Relations: %s documents" % relations.count())

Collections in database: entities, relations
Entities: 26380 documents
Relations: 32137 documents


In [6]:
# Load a little script with a couple of nice tools that developed while working
etools = MongoHelper(entities)   # Will execute on the entities database
rtools = MongoHelper(relations)  # Will execute on the relations database

# The entities collection

Als erstes werfen wir einen Blick in das erste Dokument der Collection um eine Ahnung zu erhalten, womit wir es überhaupt zu tun haben. Die Dokumente in einer MongoDB werden im JSON-Format abgespeichert.

In [6]:
# Get a sample document to have a look at and print it
first_entity_document = entities.find_one()
pprint(first_entity_document)

{u'_id': ObjectId('54bd3c748b934da06340f4c1'),
 u'aliases': [u'DIE LINKE',
              u'Die Linke',
              u'Partei DIE LINKE',
              u'DIE LINKE.',
              u'Linkspartei',
              u'Linkspartei.PDS',
              u'PDS',
              u'WASG',
              u'Partei des Demokratischen Sozialismus',
              u'Arbeit & soziale Gerechtigkeit \u2013 Die Wahlalternative',
              u'Wahlalternative',
              u'Wahlalternative Arbeit und soziale Gerechtigkeit'],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 807000),
 u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'desc': u'Partei',
            u'format': u'string',
            u'id': u'2b1adb60a31d37cf9cc0fdccb75149456a425095c6bf5e77abac117ed1a69d0f',
            u'key': u'partei',
            u'updated': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'value': u'Die Linke'},
           {u'auto': True,

In [7]:
first_entity_document['_id']

ObjectId('54bd3c748b934da06340f4c1')

In [8]:
# See what CDU looks like
#pprint(list(entities.find({"aliases": {"$eq": "CDU"}})))
pprint(list(entities.find({"name":"SPD"})))

[{u'_id': ObjectId('552ff9cdaf9ee96e1c1df7c0'),
  u'aliases': [],
  u'created': datetime.datetime(2015, 4, 16, 18, 4, 59, 967000),
  u'data': [{u'auto': True,
             u'created': datetime.datetime(2015, 5, 28, 18, 11, 10, 90000),
             u'desc': u'Partei',
             u'format': u'string',
             u'id': u'47f00e8c0c72caf8c1c4b3b5f0141df8755d5fb32f4b7f718857e45c58aff865',
             u'key': u'partei',
             u'updated': datetime.datetime(2015, 5, 28, 18, 11, 10, 90000),
             u'value': u'SPD'}],
  u'importer': u'seitenwechsler',
  u'name': u'SPD',
  u'search': [u'spd'],
  u'slug': u'spd',
  u'tags': [u'partei'],
  u'type': u'entity',
  u'updated': datetime.datetime(2015, 5, 28, 18, 11, 12, 860000)}]


In [76]:
pprint(entities.find_one({"name": "Lars Hille"}))

{u'_id': ObjectId('54db7bca28aa80ca5d49577b'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 2, 11, 15, 56, 58, 75000),
 u'data': [],
 u'importer': None,
 u'name': u'Lars Hille',
 u'search': [u'lars hille'],
 u'slug': u'lars hille',
 u'tags': [u'bank'],
 u'type': u'person',
 u'updated': datetime.datetime(2015, 2, 11, 15, 56, 58, 75000)}


### Gregor Gysi as example

In [79]:
gysi = entities.find_one({"name": "Gregor Gysi"})
gysi.get("_id")

ObjectId('54c2a4b4fe6a42c82bbab01c')

In [10]:
gysi_relations = relations.find({"entities": {"$eq": gysi.get("_id")}})
pprint(list(gysi_relations))

[{u'_id': ObjectId('54c2a4f6fe6a42c82bbabc0b'),
  u'created': datetime.datetime(2015, 1, 23, 19, 45, 58, 459000),
  u'data': [{u'auto': True,
             u'created': datetime.datetime(2015, 1, 23, 19, 45, 58, 433000),
             u'desc': u'Parteispende',
             u'format': u'donation',
             u'id': u'3321f46291fbf0dbe70a3450274aa1da3e5bcdfc75dded5358f3fe5e803e0f54',
             u'importer': u'Parteispenden',
             u'key': u'donation',
             u'updated': datetime.datetime(2015, 1, 23, 19, 45, 58, 433000),
             u'value': {u'amount': 11938.62,
                        u'sources': [u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'],
                        u'year': 1994}},
            {u'auto': True,
             u'created': datetime.datetime(2015, 1, 23, 19, 48, 2, 138000),
             u'desc': u'Parteispende',
             u'format': u'donation',
             u'id': u'844d9989c48677087381316fc07428c57d45096f31daaaf7

Die direkten Felder des Dokuments sind (abgesehen von der Id):
 _aliases, created, data, importer, name, search, slug, tags, type, updated_
 
 Zur anschließenden Analyse wurde das Tool __Variety__ verwendet (https://github.com/variety/variety). Es hilft dabei, die Daten und ihr Struktur zu verstehen und einen Überblick über sie zu bekommen.

In [11]:
# We need to find the tool variety before we can use it
VARIETY = "../variety/variety.js"
if not exists(VARIETY):
     print("variety.js script NICHT in %s gefunden. Ausversehen verschoben?" % VARIETY)
        
# Save database analysis as JSON for processing 
shell_capture = ! mongo --quiet lobbyradar --eval "var collection = 'entities', outputFormat='json'" {VARIETY}
variety_schema_json = json.loads(shell_capture.n)

# And print as ascii
! mongo --quiet lobbyradar --eval "var collection = 'entities', outputFormat='ascii'" {VARIETY}

+------------------------------------------------------------------------------------------------+
| key                      | types                        | occurrences | percents               |
| ------------------------ | ---------------------------- | ----------- | ---------------------- |
| _id                      | ObjectId                     |       26380 | 100.000000000000000000 |
| aliases                  | Array                        |       26380 | 100.000000000000000000 |
| created                  | Date                         |       26380 | 100.000000000000000000 |
| data                     | Array                        |       26380 | 100.000000000000000000 |
| importer                 | String,null                  |       26380 | 100.000000000000000000 |
| name                     | String                       |       26380 | 100.000000000000000000 |
| search                   | Array                        |       26380 | 100.000000000000000000 |
| slug    

## Hier einmal alle Felder aufgelistet (mit Typ)

In [12]:
fields = []
for x in variety_schema_json:
    fields.append( {'key': x['_id']['key'], 'type': x['value']['types']} )
pprint(fields)

[{'key': u'_id', 'type': [u'ObjectId']},
 {'key': u'aliases', 'type': [u'Array']},
 {'key': u'created', 'type': [u'Date']},
 {'key': u'data', 'type': [u'Array']},
 {'key': u'importer', 'type': [u'String', u'null']},
 {'key': u'name', 'type': [u'String']},
 {'key': u'search', 'type': [u'Array']},
 {'key': u'slug', 'type': [u'String']},
 {'key': u'tags', 'type': [u'Array']},
 {'key': u'type', 'type': [u'String']},
 {'key': u'updated', 'type': [u'Date']},
 {'key': u'data.XX.auto', 'type': [u'Boolean']},
 {'key': u'data.XX.created', 'type': [u'Date']},
 {'key': u'data.XX.desc', 'type': [u'String']},
 {'key': u'data.XX.format', 'type': [u'String']},
 {'key': u'data.XX.id', 'type': [u'String']},
 {'key': u'data.XX.key', 'type': [u'String']},
 {'key': u'data.XX.updated', 'type': [u'Date']},
 {'key': u'data.XX.value',
  'type': [u'Boolean', u'Number', u'Object', u'String']},
 {'key': u'data.XX.value.url', 'type': [u'String']},
 {'key': u'data.XX.value.remark', 'type': [u'String']},
 {'key': u'

Anschließend erzeugen wir noch eine Liste von Felder-Keys mit index um uns anhand dieser durch die Dokumente zu hangeln.

In [13]:
leKeys = []
for x in variety_schema_json:
    leKeys.append("%s : %s" % (x['_id']['key'], x['value']['types']))
indexFields = zip(range(1,40), leKeys)

In [14]:
def sample_field(field):
    return entities.find_one({ field: {"$exists": 1, "$ne": None, "$not": {"$size": 0} } }, { '_id': 0, field: 1} )

## (1, u'_id')
Die ObjectId, klar

In [15]:
sample_field("_id")

{u'_id': ObjectId('54bd3c748b934da06340f4c1')}

## (2, u'aliases'),
Array von aliases, also strings.

In [16]:
pprint(sample_field(u'aliases'))

{u'aliases': [u'"InCharge"']}


## (3, u'created'),
Zeitpunkt, als das Dokument erstellt wurde.

In [17]:
pprint(sample_field('created'))

{u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 807000)}


## (5, u'importer'),
Ich weiß bis heute nicht was damit gemeint ist!!!

In [18]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(group="importer", sort=True) )
printCols( [ formatRow(row) for row in tags ])

16804: lobbyliste             2914: parteispenden           2046: bundestag               1767: pr                      1063: None                    
541: dax                      370: laender                  311: parteispenden13          309: thinktanks               214: seitenwechsler           
34: kabinette                 6: parteispenden14            1: parteien                   


## (6, u'name'),
Achtung Sonderzeichen?

In [19]:
pprint(sample_field('name'))

{u'name': u'1. FC K\xf6ln GmbH & Co. KGaA'}


## (7, u'search'),
Wieder ein array von Strings (searchtags)

In [20]:
pprint(sample_field('search'))

{u'search': [u'1 fc koln gmbh and co kgaa', u'1 fc koln']}


## (8, u'slug'),

In [21]:
pprint(sample_field('slug'))

{u'slug': u'1 fc koln gmbh and co kgaa'}


## (9, u'tags'),

In [22]:
pprint(sample_field('tags'))

{u'tags': [u'lobbyorganisation', u'lobbyismus', u'Architektur', u'Bau']}


In [23]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="tags", group="tags", sort=True) )
printCols( [ formatRow(row) for row in tags ])

16998: lobbyismus             14740: lobbyist               13095: executive              3164: parteispenden           2222: lobbyorganisation       
1915: representative          1874: nebeneinkuenfte         979: parteispenden13          636: dax                      630: bundestag                
629: mdb                      433: aufsichtsrat             340: laender                  332: landesregierung          330: thinktank                
288: seitenwechsler           188: vorstand                 162: politik                  150: verwaltung               150: kabinette                
149: Bau                      135: Banken / Versicherungen  133: Finanzen                 126: Verwaltung / Politik     118: akademisch               
118: Energie / Wasser / Versorgung112: Industrie                100: Gesundheit               93: cdu                       92: spd                       
87: bank                      86: Immobilien                83: Handel                    

## (10, u'type'),

In [24]:
entities.distinct("type")

[u'entity', u'person']

## (11, u'updated'),

In [25]:
pprint(sample_field('updated'))

{u'updated': datetime.datetime(2015, 5, 28, 18, 11, 12, 646000)}


# Data

## (4, u'data'),
Nur ein Array

## (12, u'data.XX.auto'),

In [26]:
printCols( [formatRow(r) for r in entities.aggregate( make_pipeline(unwind="data", group="data.auto", sort=True) )] )

78515: True                   997: False                    


## (13, u'data.XX.created'),

In [27]:
pprint(sample_field('data.created'))

{u'data': [{u'created': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000)},
           {u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000)}]}


## (14, u'data.XX.desc'),

In [28]:
printCols( [formatRow(r) for r in entities.aggregate( make_pipeline(unwind="data", group="data.desc", sort=True) )] , columns=1)

27828: Quelle                 
14615: Titel                  
12712: Adresse                
5433: Link                    
3445: Nachname                
3441: Vornamen                
2433: Foto                    
2370: Beschreibungstext       
2118: Anzahl der Mitglieder   
953: Anzahl der Mitgliedsorganisationen
684: Thema                    
629: Bundesland               
608: Benutzename BT-Cert      
524: Wahlkreis                
449: Verifiziert              
402: Anzeigename              
206: URL                      
194: Landesliste              
151: Finanzierung             
130: Partei                   
126: Anzahl der Mitarbeiter   
28: Rechtsform                
14: Beschreibung              
9: Webseite                   
5: Von                        
4: Bis                        
1: Listenplatz                



In [29]:
pprint(entities.find_one({ 'data.desc': 'Quelle' }, {'_id': 0, 'data': 1}))

{u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'9f8cdd62674e492bfbf99f1950696edf49aec861cee09b275fc227a3dd4c0d97',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'remark': u'created by lobbyliste importer',
                       u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'0191e4e640931e690d1f700a4334dd24be8a57a5284883892b74f4462af77970',
            u'key': u'address',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'addr': u'Hinter der katholischen Kirche 3',
    

In [30]:
pprint(entities.find_one({ 'data.desc': 'Link' }, {'_id': 0 }))

{u'aliases': [u'Rudolf Henke'],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 47, 518000),
 u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 5, 28, 18, 11, 3, 125000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'aa8d8119eda9b934f1ccd876738a1d39c1e45c0042dd210aeb343456c0c99bfa',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 5, 28, 18, 11, 3, 125000),
            u'value': {u'remark': u'Die Welt - Spenden von Personen und Beitr\xe4ge von Mandatstr\xe4gern',
                       u'url': u'https://docs.google.com/spreadsheets/d/1caESI467tBJ8uwv0RqjI-3AJYYcrdceJhfRPAtm2yXw/edit?usp=sharing'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 5, 28, 18, 11, 3, 125000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'f78776ce755a9b930d72ca6efca58e4274c797ced24d5c2bd5577857a34159cd',
            u'key': u'address',
       

In [31]:
pprint(entities.find_one({ 'data.desc': 'Titel' }, {'_id': 0, 'data': 1}))

{u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'3dc1416e38deac59076cd3f0d4e1235de79cb530207d06c28053134cc8aa7732',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
            u'value': {u'remark': u'created by lobbyliste importer',
                       u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
            u'desc': u'Titel',
            u'format': u'string',
            u'id': u'65873d29bd0ab6af91ef341689d28f4f0658cc851494a0ef34cfd07dd0cf5d42',
            u'key': u'titles',
            u'updated': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
            u'value': u'Gesch\xe4ftsf\xfchrer'},
           {u'auto': True,
     

In [32]:
pprint(entities.find_one({ 'data.desc': 'Adresse' }, {'_id': 0, 'data': 1}))

{u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'desc': u'Partei',
            u'format': u'string',
            u'id': u'2b1adb60a31d37cf9cc0fdccb75149456a425095c6bf5e77abac117ed1a69d0f',
            u'key': u'partei',
            u'updated': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'value': u'Die Linke'},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'bb4b34d6b3f916cea2ca09b7cea7842afbec6c9d303c3dc31a5823bda70658bb',
            u'key': u'address',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000),
            u'value': {u'addr': u'Bundesgesch\xe4ftsstelle',
                       u'city': u'Berlin',
                       u'country': u'Deutschland',
                       u'email': u'bundesgeschaeftsstelle@die-linke.de',
      

In [33]:
pprint(entities.find_one({ 'data.desc': 'Quelle' }, {'_id': 0, 'data': 1}))

{u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'9f8cdd62674e492bfbf99f1950696edf49aec861cee09b275fc227a3dd4c0d97',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'remark': u'created by lobbyliste importer',
                       u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'0191e4e640931e690d1f700a4334dd24be8a57a5284883892b74f4462af77970',
            u'key': u'address',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'addr': u'Hinter der katholischen Kirche 3',
    

In [34]:
pprint(entities.find_one({ 'data.desc': 'Quelle' }, {'_id': 0, 'data': 1}))

{u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'9f8cdd62674e492bfbf99f1950696edf49aec861cee09b275fc227a3dd4c0d97',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'remark': u'created by lobbyliste importer',
                       u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'0191e4e640931e690d1f700a4334dd24be8a57a5284883892b74f4462af77970',
            u'key': u'address',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'addr': u'Hinter der katholischen Kirche 3',
    

In [35]:
p = [ 
 {'$unwind': '$data'},
 {'$match': {'data.desc': 'Wahlkreis'}},    
 {'$group': {'_id': '$data.format', 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}} 
]
result = entities.aggregate(p, cursor={})

pprint(list(result))

[{u'_id': u'string', u'count': 524}]


## (15, u'data.XX.format'),

In [36]:
printCols( [formatRow(r) for r in entities.aggregate( make_pipeline(unwind="data", group="data.format", sort=True) )] )

33277: link                   27235: string                 12712: address                3197: number                  2433: photo                   
449: bool                     199: url                      9: date                       1: integer                    


## (16, u'data.XX.id'),

In [37]:
entities.find_one()

{u'_id': ObjectId('54bd3c748b934da06340f4c1'),
 u'aliases': [u'DIE LINKE',
  u'Die Linke',
  u'Partei DIE LINKE',
  u'DIE LINKE.',
  u'Linkspartei',
  u'Linkspartei.PDS',
  u'PDS',
  u'WASG',
  u'Partei des Demokratischen Sozialismus',
  u'Arbeit & soziale Gerechtigkeit \u2013 Die Wahlalternative',
  u'Wahlalternative',
  u'Wahlalternative Arbeit und soziale Gerechtigkeit'],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 807000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
   u'desc': u'Partei',
   u'format': u'string',
   u'id': u'2b1adb60a31d37cf9cc0fdccb75149456a425095c6bf5e77abac117ed1a69d0f',
   u'key': u'partei',
   u'updated': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
   u'value': u'Die Linke'},
  {u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000),
   u'desc': u'Adresse',
   u'format': u'address',
   u'id': u'bb4b34d6b3f916cea2ca09b7cea7842afbec6c9d303c3dc31a5823bda70658bb',
   u'ke

In [38]:
pprint(sample_field('data.id'))

{u'data': [{u'id': u'2b1adb60a31d37cf9cc0fdccb75149456a425095c6bf5e77abac117ed1a69d0f'},
           {u'id': u'bb4b34d6b3f916cea2ca09b7cea7842afbec6c9d303c3dc31a5823bda70658bb'}]}


## (17, u'data.XX.key'),

In [39]:
printCols( [formatRow(r) for r in entities.aggregate( make_pipeline(unwind="data", group="data.key", sort=True) )] )

27828: source                 14560: titles                 12712: address                5433: link                    3445: surname                 
3441: names                   2433: photo                   2384: description             2118: members                 953: organisations            
684: topic                    629: bundesland               608: btcertuid                524: wahlkreis                449: verified                 
402: displayname              206: url                      194: landesliste              151: finance                  130: partei                   
126: staff                    55: title                     28: legalform                 9: www                        5: begin                      
4: end                        1: listenplatz                


## (18, u'data.XX.updated'),

In [40]:
pprint(sample_field('data.updated'))

{u'data': [{u'updated': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000)},
           {u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000)}]}


## (19, u'data.XX.value'),

In [41]:
pprint(sample_field('data'))

{u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'desc': u'Partei',
            u'format': u'string',
            u'id': u'2b1adb60a31d37cf9cc0fdccb75149456a425095c6bf5e77abac117ed1a69d0f',
            u'key': u'partei',
            u'updated': datetime.datetime(2015, 5, 28, 18, 11, 9, 657000),
            u'value': u'Die Linke'},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'bb4b34d6b3f916cea2ca09b7cea7842afbec6c9d303c3dc31a5823bda70658bb',
            u'key': u'address',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 808000),
            u'value': {u'addr': u'Bundesgesch\xe4ftsstelle',
                       u'city': u'Berlin',
                       u'country': u'Deutschland',
                       u'email': u'bundesgeschaeftsstelle@die-linke.de',
      

## (20, u'data.XX.value.url'),

In [42]:
pprint(sample_field('data.value.url'))

{u'data': [{u'value': {u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}}]}


## (21, u'data.XX.value.remark'),

In [43]:
pprint(sample_field('data.value.remark'))

{u'data': [{u'value': {u'remark': u'created by lobbyliste importer'}}]}


In [44]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.remark", sort=True) )
printCols( [ formatRow(row) for row in tags ])

46253: None                   16942: created by lobbyliste importer3197: created by bundestag importer3164: created by parteispenden importer2170: created by pr importer  
1053:                         832: Biografie auf Bundestag.de815: Die Welt - Spenden von Personen und Beiträge von Mandatsträgern726: created by dax importer  649: Abgeordnetenwatch        
630: Wikipedia                628: Fraktion                 599: Persönliche Webseite     498: Facebook-Profil          288: created by seitenwechsler importer
228: Twitter-Account          194: Reden-Archiv auf Bundestag.de178: created by thinktank importer164: Die Welt - Spenden von Unternehmen und Verbänden153: created by tinktank importer
76: YouTube-Kanal             27: Webseite                  13: Xing-Profile              4: Bundestag Drs. 18/42       3: Bundestag Drs. 17/14625    
3: Bundestag Drs. 17/14829    3: MeinVZ-Eintrag             2: Bundestag Drs. 17/14442    2: Bundestag Drs. 18/335      2: Bundestag Drs. 17/1

## (22, u'data.XX.value.name'),

In [45]:
pprint(sample_field('data.value.name'))

{u'data': [{u'value': {u'name': u'Forschungsforum "\xd6ffentliche Sicherheit" e.V.'}}]}


In [46]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.name", sort=True) )
printCols( [ formatRow(row) for row in tags ])

67133: None                   133: Hausadresse:             19: Kirsten Tackmann          11: Jutta Krellmann           11: Simone Raatz              
10: Frank Schwabe             10: Kerstin Andreae           10: Cem Özdemir               10: Jürgen Trittin            10: Frank Steffel             
10: Claudia Roth              10: Bärbel Höhn               9: Kathrin Vogler             9: Thomas Nord                9: Heidrun Bluhm              
9: Südzucker AG               9: Dietmar Bartsch            9: Renate Künast              9: Volker Beck                9: Bundesverband Deutscher Vermögensberater e.V.
9: Niema Movassat             8: Frank Tempel               8: Ralph Lenkert              8: Christine Buchholz         8: Sabine Leidig              
8: Inge Höger                 8: Gerhard Schick             8: Omid Nouripour             8: Kerstin Griese             8: Roland Claus               
8: Verband der Chemischen Industrie e.V.8: Waltraud Wolff             8: Kar

## (23, u'data.XX.value.addr'),

In [47]:
pprint(sample_field('data.value.addr'))

{u'data': [{u'value': {u'addr': u'Carl-Heinrich-Becker-Weg 6-10'}}]}


## (24, u'data.XX.value.city'),

In [48]:
pprint(sample_field('data.value.city'))

{u'data': [{u'value': {u'city': u'Berlin'}}]}


## (25, u'data.XX.value.postcode'),

In [49]:
pprint(sample_field('data.value.postcode'))

{u'data': [{u'value': {u'postcode': u'12165'}}]}


## (26, u'data.XX.value.type'),

In [50]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.type", sort=True) )
printCols( [ formatRow(row) for row in tags ])

66819: None                   11939: main                   754: extra                    


## (27, u'data.XX.value.country'),

In [51]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.country", sort=True) )
printCols( [ formatRow(row) for row in tags ])

69378: None                   5054: Germany                 2924: Deutschland             2071:                         44: Belgien                   
15: Switzerland               5: USA                        4: United Kingdom             3: Austria                    3: Schweiz                    
2: Niederlande                1:  Türkei                    1: Russland                   1:  Österreich                1: Israel                     
1: Vereinigtes Königreich     1: Cyprus                     1: Luxemburg                  1: Polen                      1:  Belgien                   



## (28, u'data.XX.value.street'),

In [52]:
pprint(sample_field('data.value.street'))

{u'data': [{u'value': {u'street': u'Friedrichstr. 136'}}]}


## (29, u'data.XX.value.tel'),

In [53]:
pprint(sample_field('data.value.tel'))

{u'data': [{u'value': {u'tel': u'+49 (0)30 838 57367'}}]}


## (30, u'data.XX.value.email'),

In [54]:
pprint(sample_field('data.value.email'))

None


Existiert aber. String, falls email vorhanden

## (31, u'data.XX.value.fax'),

In [55]:
pprint(sample_field('data.value.fax'))

{u'data': [{u'value': {u'fax': u'+49 (0)30 838 4 57367'}}]}


## (32, u'data.XX.value.www'),

In [56]:
pprint(sample_field('data.value.www'))

{u'data': [{u'value': {u'www': u'http://www.sicherheit-forschung.de/'}}]}


## (33, u'data.XX.value.copyright'),

In [57]:
pprint(sample_field('data.value.copyright'))

None


In [58]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.copyright", sort=True, min_count=50) )
printCols( [ formatRow(row) for row in tags ])

77207: None                   276:                          175: Foto-AG Gymnasium Melle  171: © spdfraktion.de         104: Indeedous                
72: Bildquelle: Laurence Chaperon69: Steschke                  58: © SPD Parteivorstand / Susie Knoll / Florian Jaenicke 56: © DIE LINKE im Bundestag  54: © gruene-bundestag.de     



## (34, u'data.XX.value.county'),

In [59]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.county", sort=True) )
printCols( [ formatRow(row) for row in tags ])

78279: None                   1233: Germany                 


## (35, u'data.XX.value.license'),

In [60]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.license", sort=True) )
printCols( [ formatRow(row) for row in tags ])

78728: None                   234: http://creativecommons.org/licenses/by-sa/3.0/de/199: CC-BY-SA-3.0-DE          187: http://creativecommons.org/licenses/by/3.0/legalcode87: http://creativecommons.org/licenses/by-sa/3.0/legalcode
27: CC-BY-SA-4.0              17: CC-BY-2.0                 10: CC-BY-SA-2.0              5: CC-BY-SA-3.0,2.5,2.0,1.0   3: CC-BY-SA-3.0-AT            
2: CC-BY-3.0-DE               2: PD                         2: http://www.gnu.org/licenses/fdl.txt2: CC-BY-SA-2.0-DE            1: CC-BY-4.0                  
1: CC-Zero                    1: PD-user                    1: CC-BY-SA-3.0-migrated      1: http://de.wikipedia.org/wiki/Datei:Dietmar_Nietan_MdB,_2013.jpg1: http://creativecommons.org/licenses/by-sa/2.5/legalcode
1: PD US Military             


## (36, u'data.XX.value.source_url'),

In [61]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="data", group="data.value.source_url", sort=True) )
printCols( [ formatRow(row) for row in tags ])

78962: None                   2: http://de.wikipedia.org/wiki/Datei:Die_1968_in_Wittingen_geborene_SPD-Politikerin_Kerstin_Tack_ist_u.a._seit_2009_Mitglied_des_Deutschen_Bundestages_(MdB)_und_seit_2012_Vorsitzende_des_Bezirksverbandes_Hannover_der_Arbeiterwohlfahrt.jpg1: http://de.wikipedia.org/wiki/Datei:WLP14-ri-0279-_Brigitte_Zypries_(SPD).jpg1: http://de.wikipedia.org/wiki/Datei:Zollner,_Gudrun-9469.jpg1: http://de.wikipedia.org/wiki/Datei:Zimmermann,_Pia-1246.jpg
1: http://de.wikipedia.org/wiki/Datei:Zimmer,_Matthias-0623.jpg1: http://de.wikipedia.org/wiki/Datei:WLP14-ri-0401-_Stefan_Zierke_(SPD),_MdB.jpg1: http://de.wikipedia.org/wiki/Datei:Emmi_Zeulner_2013.jpg1: http://de.wikipedia.org/wiki/Datei:Zertik,_Heinrich-0584.jpg1: http://de.wikipedia.org/wiki/Datei:Tobias_Zech_2009.jpg
1: http://de.wikipedia.org/wiki/Datei:WLP14-ri-0720-_J%C3%B6rn_Wunderlich_(Die_Linke).jpg1: http://de.wikipedia.org/wiki/Datei:Barbara_Woltmann_-6802.jpg1: http://de.wikipedia.org/wiki/Datei:MK19814_Hei

## (37, u'data.XX.value.date'),

In [62]:
pprint(sample_field('data.value.date'))

{u'data': [{u'value': {u'date': datetime.datetime(2013, 12, 16, 23, 0)}}]}


## (38, u'data.XX.value.fmt')]

In [63]:
pprint(sample_field('data.value.fmt'))

{u'data': [{u'value': {u'fmt': u'dd.MM.yyyy'}}]}


## Tags

Da Tags ein Array ist, muss es in der Pipeline zunächst in seine Elemente aufgeteilt werden.

In [64]:
# Aggregation, die das "tags"-Array der Dokumente zerlegt und für jeden Wert einen neuen Eintrag im Resultset erzeugt.
# Diese werden anschließend gruppiert, die Gruppengrößen gezählt und entsprechend sortiert
tags = entities.aggregate( make_pipeline(unwind="tags", group="tags", sort=True) )
printCols( [ formatRow(row) for row in tags ])

16998: lobbyismus             14740: lobbyist               13095: executive              3164: parteispenden           2222: lobbyorganisation       
1915: representative          1874: nebeneinkuenfte         979: parteispenden13          636: dax                      630: bundestag                
629: mdb                      433: aufsichtsrat             340: laender                  332: landesregierung          330: thinktank                
288: seitenwechsler           188: vorstand                 162: politik                  150: verwaltung               150: kabinette                
149: Bau                      135: Banken / Versicherungen  133: Finanzen                 126: Verwaltung / Politik     118: akademisch               
118: Energie / Wasser / Versorgung112: Industrie                100: Gesundheit               93: cdu                       92: spd                       
87: bank                      86: Immobilien                83: Handel                    

In [65]:
# Get an example of tag "lobbyismus"
pprint(list(db.entities.find({ "tags": {"$eq": "parteispenden"}}).limit(5)))

[{u'_id': ObjectId('54bd3c768b934da06340f50a'),
  u'aliases': [],
  u'created': datetime.datetime(2015, 1, 19, 17, 18, 46, 830000),
  u'data': [{u'auto': True,
             u'created': datetime.datetime(2015, 1, 23, 19, 44, 50, 892000),
             u'desc': u'Quelle',
             u'format': u'link',
             u'id': u'870f0255a688738d3f1cbac751e60f4f48e0cac85842ce881dbdbabc8e502d58',
             u'key': u'source',
             u'updated': datetime.datetime(2015, 1, 23, 19, 44, 50, 892000),
             u'value': {u'remark': u'created by parteispenden importer',
                        u'url': u'http://apps.opendatacity.de/parteispenden-recherche/assets/data/parteispenden.json'}},
            {u'auto': True,
             u'created': datetime.datetime(2015, 1, 23, 19, 44, 50, 892000),
             u'desc': u'Adresse',
             u'format': u'address',
             u'id': u'7e3ebbe597747c947566f51876b736521b64daf724df9e0542b7b1cc0b355488',
             u'key': u'address',
        

## Type

In [66]:
# Aggregation mit Gruppierung der Dokumente nach Feld "type", zählen der Gruppen und aufsteigend danach sortieren
pipeline = [ 
    { "$group": {"_id": "$type", "count": {"$sum": 1}} },
    { "$sort": {"count":1} }
]

distinct_types_count = list(db.entities.aggregate(pipeline, cursor={}))
pprint(distinct_types_count)
print("Total: %s distinct types." % len(distinct_types_count))

[{u'_id': u'entity', u'count': 6552}, {u'_id': u'person', u'count': 19828}]
Total: 2 distinct types.


# Importer

In [67]:
# Aggregation mit Gruppierung der Dokumente nach Feld "importer", zählen der Gruppen und aufsteigend danach sortieren
pipeline = [ 
    { "$group": {"_id": "$importer", "count": {"$sum": 1}} },
    { "$sort": {"count": -1} }
]

distinct_importer_count = list(db.entities.aggregate(pipeline, cursor={}))
pprint(distinct_importer_count)
print("Total: %s distinct importer." % len(distinct_importer_count))

[{u'_id': u'lobbyliste', u'count': 16804},
 {u'_id': u'parteispenden', u'count': 2914},
 {u'_id': u'bundestag', u'count': 2046},
 {u'_id': u'pr', u'count': 1767},
 {u'_id': None, u'count': 1063},
 {u'_id': u'dax', u'count': 541},
 {u'_id': u'laender', u'count': 370},
 {u'_id': u'parteispenden13', u'count': 311},
 {u'_id': u'thinktanks', u'count': 309},
 {u'_id': u'seitenwechsler', u'count': 214},
 {u'_id': u'kabinette', u'count': 34},
 {u'_id': u'parteispenden14', u'count': 6},
 {u'_id': u'parteien', u'count': 1}]
Total: 13 distinct importer.


## Data

### data.auto

### Werte von "Desc"

In [68]:
entities.find_one({ "data.desc": "Titel"})

{u'_id': ObjectId('54bd3c768b934da06340f4c6'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 46, 529000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
   u'desc': u'Quelle',
   u'format': u'link',
   u'id': u'3dc1416e38deac59076cd3f0d4e1235de79cb530207d06c28053134cc8aa7732',
   u'key': u'source',
   u'updated': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
   u'value': {u'remark': u'created by lobbyliste importer',
    u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
  {u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
   u'desc': u'Titel',
   u'format': u'string',
   u'id': u'65873d29bd0ab6af91ef341689d28f4f0658cc851494a0ef34cfd07dd0cf5d42',
   u'key': u'titles',
   u'updated': datetime.datetime(2015, 1, 19, 17, 19, 9, 420000),
   u'value': u'Gesch\xe4ftsf\xfchrer'},
  {u'auto': True,
   u'created': datetime.datetime(2015, 

In [69]:
p = [{'$unwind': '$data'},
 {"$match": {"data.desc": "Titel"}},
 {'$group': {'_id': '$data.format', 'count': {'$sum': 1}}},
 {'$sort': {'count': -1}}]

printCols([ formatRow(r) for r in entities.aggregate(p)])

14615: string                 


### Werte von "Format"

### Werte von "Key"

In [70]:
etools.find_if_exists("data.value.url", {"data":1})

{u'_id': ObjectId('54bd3c768b934da06340f4cd'),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 58, 507000),
   u'desc': u'Quelle',
   u'format': u'link',
   u'id': u'48af1365a87b478077bd4548426cf5f1dc893a314fdb9e27b410e949337d1627',
   u'key': u'source',
   u'updated': datetime.datetime(2015, 1, 19, 17, 18, 58, 507000),
   u'value': {u'remark': u'created by lobbyliste importer',
    u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}}]}

In [71]:
etools.find_if_exists("data.value.remark", {"data":1})

{u'_id': ObjectId('54bd3c768b934da06340f4cd'),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 58, 507000),
   u'desc': u'Quelle',
   u'format': u'link',
   u'id': u'48af1365a87b478077bd4548426cf5f1dc893a314fdb9e27b410e949337d1627',
   u'key': u'source',
   u'updated': datetime.datetime(2015, 1, 19, 17, 18, 58, 507000),
   u'value': {u'remark': u'created by lobbyliste importer',
    u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}}]}

In [72]:
etools.find_if_exists("data.value.name" )

{u'_id': ObjectId('54cfa3918b4cf88d203b8a85'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 2, 2, 16, 19, 29, 349000),
 u'data': [{u'auto': False,
   u'created': datetime.datetime(2015, 2, 2, 16, 19, 29, 349000),
   u'desc': u'Adresse',
   u'format': u'address',
   u'id': u'f9e892630dd5c9f1bfdb23eb79cbe704dc0f755c646d5eead78b75dc73173b79',
   u'key': u'address',
   u'updated': datetime.datetime(2015, 2, 2, 16, 19, 29, 349000),
   u'value': {u'addr': u'Carl-Heinrich-Becker-Weg 6-10',
    u'city': u'Berlin',
    u'country': u'Deutschland',
    u'fax': u'+49 (0)30 838 4 57367',
    u'name': u'Forschungsforum "\xd6ffentliche Sicherheit" e.V.',
    u'postcode': u'12165',
    u'tel': u'+49 (0)30 838 57367',
    u'www': u'http://www.sicherheit-forschung.de/'}}],
 u'importer': None,
 u'name': u'Forschungsforum "\xd6ffentliche Sicherheit" e.V.',
 u'search': [u'forschungsforum offentliche sicherheit ev'],
 u'slug': u'forschungsforum offentliche sicherheit ev',
 u'tags': [],
 u'type': u'

In [73]:
pprint(entities.find_one({ "data.desc": {"$eq": "Anzahl der Mitglieder"}}))

{u'_id': ObjectId('54bd3c768b934da06340f4c5'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
 u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'9f8cdd62674e492bfbf99f1950696edf49aec861cee09b275fc227a3dd4c0d97',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'value': {u'remark': u'created by lobbyliste importer',
                       u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 19, 17, 18, 44, 930000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'0191e4e640931e690d1f700a4334dd24be8a57a5284883892b74f4462af77970',
            u'key': u'address',
            u'update

In [74]:
pprint(entities.find_one({ "data.desc": {"$eq": "Anzahl der Mitarbeiter"}}))

{u'_id': ObjectId('54bd3c808b934da063410df4'),
 u'aliases': [u'DGAP'],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 45, 552000),
 u'data': [{u'auto': True,
            u'created': datetime.datetime(2015, 1, 30, 13, 16, 43, 872000),
            u'desc': u'Quelle',
            u'format': u'link',
            u'id': u'9fa39ace3522476a01edc7803c68efd2bc3d99c7fe0b7c97ea24a02b2cf4a704',
            u'key': u'source',
            u'updated': datetime.datetime(2015, 1, 30, 13, 16, 43, 872000),
            u'value': {u'remark': u'created by bundestag importer',
                       u'url': u'http://www.bundestag.de/bundestag/abgeordnete18/biografien/M/missfelder_philipp/258790'}},
           {u'auto': True,
            u'created': datetime.datetime(2015, 1, 30, 13, 16, 43, 872000),
            u'desc': u'Adresse',
            u'format': u'address',
            u'id': u'b9083c019f8d8aa1a221b5318bac2a45e81736728a12aa5b353391ebff4666bc',
            u'key': u'address',
            u'updat

In [75]:
# Print an example value of each description type
for desc in distinct_desc_types:
    desc_type = desc['_id']['desc']
    entity_with_desc_type = db.entities.find_one({ "data.desc": {"$eq": desc_type} }, { "data": 1 })
    #entity_with_desc_type = db.entities.find_one({ "data.desc": {"$eq": desc_type} }, { "data": {"$elemMatch": {"data.desc": desc_type}} })
    pprint(entity_with_desc_type)
    
    

NameError: name 'distinct_desc_types' is not defined

In [7]:
db.entities.find_one({ "_id": ObjectId("54bd3c908b934da063412c8a") })

{u'_id': ObjectId('54bd3c908b934da063412c8a'),
 u'aliases': [],
 u'created': datetime.datetime(2015, 1, 19, 17, 18, 46, 320000),
 u'data': [{u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 46, 320000),
   u'desc': u'Quelle',
   u'format': u'link',
   u'id': u'3b0ababcba4069f956f07ba237409d642ecbd8967394b73fac571a17630d3296',
   u'key': u'source',
   u'updated': datetime.datetime(2015, 1, 19, 17, 18, 46, 320000),
   u'value': {u'remark': u'created by lobbyliste importer',
    u'url': u'http://bundestag.de/blob/189476/8989cc5f5f65426215d7e0233704b20a/lobbylisteaktuell-data.pdf'}},
  {u'auto': True,
   u'created': datetime.datetime(2015, 1, 19, 17, 18, 46, 320000),
   u'desc': u'Adresse',
   u'format': u'address',
   u'id': u'602be655a97de9ee7fd9d4b431cfd809c54ca664f360718d8cde866cb7a8800b',
   u'key': u'address',
   u'updated': datetime.datetime(2015, 1, 19, 17, 18, 46, 320000),
   u'value': {u'addr': u'Askanischer Platz 1',
    u'city': u'Berlin',
    u'country': u'